In [60]:
import requests as r
import networkx as nx
from bs4 import BeautifulSoup

In [80]:
def film2actors(link):
    html = r.get(link).text
    soup = BeautifulSoup(html)
    actor_sou = soup.find_all(class_ = 'cast-list')
    actors = BeautifulSoup(str(actor_sou[0])).find_all('a')
    return [x.attrs['href'] for x in actors]

In [81]:
def film2directors(link):
    html = r.get(link).text
    soup = str('\n'.join([str(x) for x in BeautifulSoup(html).find_all(class_ = 'text-sluglist')]))
    crew = BeautifulSoup(soup).find_all('a')
    return [x.attrs['href'] for x in crew if 'director' in x.attrs['href']]

In [82]:
def actor2films(link):
    html = r.get(link).text
    soup = BeautifulSoup(html).find_all(class_ = 'poster-container')
    parsed = [BeautifulSoup(str(x)).find('div') for x in soup]
    return [x.attrs['data-target-link'] for x in parsed]

In [83]:
def director2films(link):
    html = r.get(link).text
    soup = BeautifulSoup(html).find_all(class_ = 'poster-container')
    parsed = [BeautifulSoup(str(x)).find('div') for x in soup]
    return [x.attrs['data-target-link'] for x in parsed]

In [96]:
def step_on_graph(graph, discovered, current_item, verb, target_queue, explored_set):
    
    for item in discovered:
        if not item in graph:

            if current_item in graph: graph[current_item][item].append(verb) 
            else: graph[current_item] = {item: [verb]}

        else: 

            if current_item in graph[item]: graph[item][current_item].append(verb)
            else: graph[item][current_item] = [verb]

        if not item in explored_set and not item in target_queue: target_queue.append(item)
        
        
    
    

In [97]:
graph = {}
directors_queue = ['/director/alejandro-amenabar']
actors_queue = []
films_queue = []

MAX_NODES = 15000

director_fstring = "https://letterboxd.com{}"
actor_fstring = "https://letterboxd.com{}"
film_fstring = "https://letterboxd.com{}"

explored_items = set()

In [98]:
while len(explored_items) < MAX_NODES and (len(directors_queue) or len(actors_queue) or len(films_queue)):
    
    print(f"Currently explored {len(explored_items)} items. {len(films_queue)} films, {len(actors_queue)} actors and {len(directors_queue)} directors queued. \t", end = '\r')
    if len(directors_queue):
        
        current_director = directors_queue.pop(0)
        films = director2films(director_fstring.format(current_director))
        step_on_graph(graph, films, current_director, 'directed', films_queue, explored_items)
        explored_items.add(current_director)
    
    if len(actors_queue):
        
        current_actor = actors_queue.pop(0)
        films = actor2films(actor_fstring.format(current_actor))
        step_on_graph(graph, films, current_actor, 'acted', films_queue, explored_items)
        explored_items.add(current_actor)
        
    if len(films_queue):
        
        current_film = films_queue.pop(0)
        
        ### DIRECTOR DISCOVERY ###
        directors = film2directors(film_fstring.format(current_film))
        step_on_graph(graph, directors, current_film, 'directed', directors_queue, explored_items)
        
        
        ### ACTOR DISCOVERY ###
        actors = film2actors(film_fstring.format(current_film))
        step_on_graph(graph, actors, current_film, 'acted', actors_queue, explored_items)
        
        explored_items.add(current_film)
        
    
    nx.write_gexf(nx.from_dict_of_dicts(graph), './graph.gexf')
        
    
        
    

KeyError: '/film/open-your-eyes/'